In [1]:
!pip install scikit-learn==1.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 41.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 39.4 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install pandas 
!pip install psycopg2-binary


[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install sqlalchemy==1.4.45

  Using cached SQLAlchemy-1.4.45-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
def check_condition() -> bool:
    import os
    import pandas as pd
    from sqlalchemy import create_engine, text
    import boto3
    import json

    def get_secret():

        secret_name = "DBCreds"
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            raise e

        secret = get_secret_value_response['SecretString']

        # Parse the secret string to get the credentials
        secret_dict = json.loads(secret)
        username = secret_dict['username']
        password = secret_dict['password']
        host = secret_dict['host']
        port = secret_dict['port']
        dbname = secret_dict['dbname']

        return username, password, host, port, dbname


    (user,pswd,host,port,db) = get_secret()

    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }

    # Connect to PostgreSQL
    engine = create_engine(f'postgresql+psycopg2://{db_details["user"]}:{db_details["password"]}@{db_details["host"]}:{db_details["port"]}/{db_details["dbname"]}', connect_args={'connect_timeout': 60})

    try:
        with engine.connect() as conn:
            query = text('SELECT count(*) FROM intrusion_outcomes;')
            data = pd.read_sql_query(query, conn)
            count = data.iloc[0]['count']
    except Exception as e:
        print("error")
    
    try:
        with engine.connect() as conn:
            query = text('SELECT count(*) FROM metadata_table_intrusion;')
            data = pd.read_sql_query(query, conn)
            meta_count = data.iloc[0]['count']
    except Exception as e:
        print("error")

    if (count % 5 == 0 and count != 0) or meta_count == 0:
        return True
    else:
        return False

In [2]:
def read_file() -> None:
    import os
    import pandas as pd
    import numpy as np
    from minio import Minio
    from scipy.special import boxcox
    from sklearn.model_selection import train_test_split
    import boto3
    import json
    import pickle
    
    import base64
    import psycopg2
    from psycopg2 import sql
    from sqlalchemy import create_engine, text
    import datetime
    
    def get_secret():

        secret_name = "DBCreds"
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            raise e

        secret = get_secret_value_response['SecretString']
    
        # Parse the secret string to get the credentials
        secret_dict = json.loads(secret)
        username = secret_dict['username']
        password = secret_dict['password']
        host = secret_dict['host']
        port = secret_dict['port']
        dbname = secret_dict['dbname']

        return username, password, host, port, dbname


    (user,pswd,host,port,db) = get_secret()
    
    # Dictionary to save mean, sd, and, encoder
    preprocess_df = {'version':1}
    
    #Perform normalization
    def zscore_normalization(df, name):
        mean = df[name].mean()
        sd = df[name].std()
        df[name] = (df[name] - mean) / sd
        preprocess_df[name] = (mean, sd)
        
    #Encode text 
    def encode_text(df, name):
        from sklearn.preprocessing import OrdinalEncoder
        enc = OrdinalEncoder()
        data = enc.fit_transform(df[name].values.reshape(-1,1))
        df[name] = data.flatten()
        preprocess_df[name] = base64.b64encode(pickle.dumps(enc)).decode('utf-8')
        # pickle.loads(a.encode('latin1'))
        
    #Data preprocessing
    def preprocess(df):

        for col in df.columns:
            t = (df[col].dtype)
            if col != 'outcome':
                if (t == int or t == float):
                    zscore_normalization(df, col)
               
                else:
                    df[col] = df[col].astype(str)
                    encode_text(df, col)
                            
        for col in df.columns:
            if len(df[col].unique()) == 1:
                df.drop(col, inplace=True,axis=1)
                preprocess_df[col] = None

        #df.loc[df['outcome'] != "normal.", 'outcome']  = 1
        #df.loc[df['outcome'] == "normal.", 'outcome']  = 0
        #encode_text(df, "outcome")
        print(df.columns)
        correlation = df.corrwith(df["outcome"])
      
        
        row = 0 
        for num in correlation:
            if num >= -0.05 and num <= 0.05:
                preprocess_df[df.columns[row]] = None
                df.drop(df.columns[row], axis=1, inplace=True)
                row -= 1
            row += 1
        return df
    
    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
    
    # Connect to PostgreSQL
    engine = create_engine(f'postgresql+psycopg2://{db_details["user"]}:{db_details["password"]}@{db_details["host"]}:{db_details["port"]}/{db_details["dbname"]}', connect_args={'connect_timeout': 60})
    
    df = pd.DataFrame()
    
    try:
        with engine.connect() as conn:
            query = text('SELECT * FROM intrusion_data WHERE outcome != 2;')
            chunksize = 10000  # Adjust chunksize as per your memory and performance needs
            chunks = pd.read_sql_query(query, conn, chunksize=chunksize)
            i = 1
            for chunk in chunks:
                # print(f"chunk{i}")
                i = i + 1
                features_df = pd.json_normalize(chunk['features'])
                features_df['outcome'] = chunk['outcome']
                df = pd.concat([df, features_df], ignore_index=True)

                # Get current process (notebook process) ID
                pid = os.getpid()
                process = psutil.Process(pid)

                # Get CPU and memory usage
#                 cpu_percent = process.cpu_percent()
#                 mem_info = process.memory_info()
#                 mem_usage = mem_info.rss / (1024 * 1024)  # in MiB

#                 print(f"CPU Usage: {cpu_percent}%")
#                 print(f"Memory Usage: {mem_usage} MiB")
                
                
                
    except Exception as e:
            print(f"Failed to fetch data: {e}")

    print(df.columns)
    
    #df = df.drop(columns=['timestamp','uid'])
    df = preprocess(df)
    X = df.drop(columns=['outcome'])
    y = df['outcome']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    bucket_name="intrusionpipeline"
    role_arn = 'arn:aws:iam::533267059960:role/aws-s3-access'
    session_name = 'kubeflow-pipeline-session'
    sts_client = boto3.client('sts')
    response = sts_client.assume_role(RoleArn=role_arn, RoleSessionName=session_name)
    credentials = response['Credentials']
    # Configure AWS SDK with temporary credentials
    s3_client = boto3.client('s3',
                      aws_access_key_id=credentials['AccessKeyId'],
                      aws_secret_access_key=credentials['SecretAccessKey'],
                      aws_session_token=credentials['SessionToken'])
    
    
    
    folder_path = './tmp/intrusion'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")
        

    try:
        with engine.connect() as conn:
            query = text('SELECT * FROM metadata_table_intrusion ORDER BY version DESC LIMIT 1;')
            data = pd.read_sql_query(query, conn)
            version = data['version'].iloc[0] + 1
    except Exception as e:
        version = 1
        
    df.to_csv("./tmp/intrusion/intrusion_data.csv")
    s3_client.upload_file("./tmp/intrusion/intrusion_data.csv", bucket_name, f"version{version}/intrusion_dataset.csv")
    np.save("./tmp/intrusion/X_train.npy",X_train)
    s3_client.upload_file("./tmp/intrusion/X_train.npy", bucket_name, f"version{version}/X_train.npy")
    np.save("./tmp/intrusion/y_train.npy",y_train)
    s3_client.upload_file("./tmp/intrusion/y_train.npy", bucket_name, f"version{version}/y_train.npy")
    np.save("./tmp/intrusion/X_test.npy",X_test)
    s3_client.upload_file("./tmp/intrusion/X_test.npy", bucket_name, f"version{version}/X_test.npy")
    np.save("./tmp/intrusion/y_test.npy",y_test)
    s3_client.upload_file("./tmp/intrusion/y_test.npy", bucket_name, f"version{version}/y_test.npy")
        
        
    preprocess_df['version'] = version
    mean_df = pd.DataFrame([preprocess_df])
    meta_df = pd.DataFrame(data = [[version, datetime.datetime.now(), len(X.columns), json.dumps(df.dtypes.astype(str).to_dict()),mean_df.iloc[0].to_json()]], columns = ['version', 'date', 'features', 'types','factor'])
    meta_df.to_sql("metadata_table_intrusion", engine, if_exists='append', index=False)

In [17]:
read_file()

Index(['hot', 'flag', 'land', 'count', 'urgent', 'service', 'duration',
       'num_root', 'dst_bytes', 'logged_in', 'src_bytes', 'srv_count',
       'num_shells', 'root_shell', 'rerror_rate', 'serror_rate',
       'su_attempted', 'diff_srv_rate', 'is_host_login', 'protocol_type',
       'same_srv_rate', 'dst_host_count', 'is_guest_login', 'wrong_fragment',
       'num_compromised', 'srv_rerror_rate', 'srv_serror_rate',
       'num_access_files', 'num_failed_logins', 'num_outbound_cmds',
       'dst_host_srv_count', 'num_file_creations', 'srv_diff_host_rate',
       'dst_host_rerror_rate', 'dst_host_serror_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_srv_rate',
       'dst_host_srv_rerror_rate', 'dst_host_srv_serror_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'outcome'],
      dtype='object')
Index(['hot', 'flag', 'land', 'count', 'urgent', 'service', 'duration',
       'num_root', 'dst_bytes', 'logged_in', 'src_bytes', 'srv_count',
    

In [3]:
def train_op() -> None:
    import pickle
    import pandas as pd
    import numpy as np
    import json
    import os
    import time
    import tensorflow as tf
    import boto3
    from minio import Minio
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.svm import SVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sqlalchemy import create_engine
    from sqlalchemy import create_engine, Table, Column, Float, Integer, String, MetaData, ARRAY
    from sqlalchemy import select, desc, insert, text
    from io import BytesIO
    
    import psycopg2
    from psycopg2 import sql
    from sqlalchemy import create_engine
    
    def get_secret():
        secret_name = "DBCreds"
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            raise e

        secret = get_secret_value_response['SecretString']
    
        # Parse the secret string to get the credentials
        secret_dict = json.loads(secret)
        username = secret_dict['username']
        password = secret_dict['password']
        host = secret_dict['host']
        port = secret_dict['port']
        dbname = secret_dict['dbname']

        return username, password, host, port, dbname


    (user,pswd,host,port,db) = get_secret()
    
    bucket_name="intrusionpipeline"
    role_arn = 'arn:aws:iam::533267059960:role/aws-s3-access'
    session_name = 'kubeflow-pipeline-session'
    sts_client = boto3.client('sts')
    response = sts_client.assume_role(RoleArn=role_arn, RoleSessionName=session_name)
    credentials = response['Credentials']
    
    # Configure AWS SDK with temporary credentials
    s3_client = boto3.client('s3',
                      aws_access_key_id=credentials['AccessKeyId'],
                      aws_secret_access_key=credentials['SecretAccessKey'],
                      aws_session_token=credentials['SessionToken'])
    
    
    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
        
        
    
    # Connect to PostgreSQL
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        exit()
        
    # Query to fetch data from the table
    try:
        fetch_query = "SELECT * FROM metadata_table_intrusion ORDER BY date DESC LIMIT 1;"
        df = pd.read_sql(fetch_query, conn)
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    if(not df.empty):
        version = df['version'][0]
    else:
        version = 1
        
    folder_path = f"version{version}"
    
    cursor.close()
    conn.close()
    
    print(f"version{version}/X_train.npy")
    
    response = s3_client.get_object(Bucket=bucket_name, Key=f"version{version}/X_train.npy")
    data = response['Body'].read()
    X_train = np.load(BytesIO(data))
    X_train = pd.DataFrame(X_train)
    
    response = s3_client.get_object(Bucket=bucket_name, Key=f"version{version}/y_train.npy")
    data = response['Body'].read()
    y_train = np.load(BytesIO(data))

    
    response = s3_client.get_object(Bucket=bucket_name, Key=f"version{version}/X_test.npy")
    data = response['Body'].read()
    X_test = np.load(BytesIO(data))
    X_test = pd.DataFrame(X_test)
    
    response = s3_client.get_object(Bucket=bucket_name, Key=f"version{version}/y_test.npy")
    data = response['Body'].read()
    y_test = np.load(BytesIO(data))
    
    
    # Define dataframe to store model metrics
    metrics = pd.DataFrame(columns=["Version", "Model", "Accuracy", "F1", "Precision", "Recall", "Train_Time", "Test_Time"])
    models_path = './tmp/intrusion/models'
    
    
    if not os.path.exists(models_path):
        os.makedirs(models_path)
        print(f"Folder '{models_path}' created successfully.")
    else:
        print(f"Folder '{models_path}' already exists.")
        
    
    #Logistic Regression
    start_train = time.time()
    lrc = LogisticRegression(random_state=0, max_iter=1000)
    lrc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    ypredlr = lrc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test, ypredlr)
    f1 = f1_score(y_test, ypredlr)
    precision = precision_score(y_test, ypredlr)
    recall = recall_score(y_test, ypredlr)
    metrics.loc[len(metrics.index)] = [version,'lrc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    with open('./tmp/intrusion/models/lrc.pkl', 'wb') as f:
        pickle.dump(lrc, f)
    s3_client.upload_file("tmp/intrusion/models/lrc.pkl", bucket_name, f"{folder_path}/lrc/model.pkl")
    
    
    #Random Forest Classifier
    start_train = time.time()
    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred2=rfc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test, y_pred2)
    f1 = f1_score(y_test, y_pred2)
    precision = precision_score(y_test, y_pred2)
    recall = recall_score(y_test, y_pred2)
    metrics.loc[len(metrics.index)] = [version, 'rfc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    with open('./tmp/intrusion/models/rfc.pkl', 'wb') as f:
        pickle.dump(rfc, f)
    s3_client.upload_file("tmp/intrusion/models/rfc.pkl", bucket_name, f"{folder_path}/rfc/model.pkl")
    
    
#     #Decision Tree
#     start_train = time.time()
#     dtc = DecisionTreeClassifier()
#     dtc.fit(X_train, y_train)
#     end_train = time.time()
#     start_test = time.time()
#     y_pred3=dtc.predict(X_test)
#     end_test = time.time()
#     accuracy = accuracy_score(y_test,y_pred3)
#     f1 = f1_score(y_test, y_pred3)
#     precision = precision_score(y_test, y_pred3)
#     recall = recall_score(y_test, y_pred3)
#     metrics.loc[len(metrics.index)] = [version, 'dtc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
#     with open('./tmp/intrusion/models/dtc.pkl', 'wb') as f:
#         pickle.dump(dtc, f)
#     s3_client.upload_file("tmp/intrusion/models/dtc.pkl", bucket_name, f"{folder_path}/dtc/model.pkl")
    
    
    
#     #Support Vector Machine
#     start_train = time.time()
#     svc = SVC()
#     svc.fit(X_train, y_train)
#     end_train = time.time()
#     start_test = time.time()
#     y_pred4=svc.predict(X_test)
#     end_test = time.time()
#     accuracy = accuracy_score(y_test,y_pred4)
#     f1 = f1_score(y_test,y_pred4)
#     precision = precision_score(y_test, y_pred4)
#     recall = recall_score(y_test, y_pred4)
#     metrics.loc[len(metrics.index)] = [version, 'svc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
#     with open('./tmp/intrusion/models/svc.pkl', 'wb') as f:
#         pickle.dump(svc, f)
#     s3_client.upload_file("tmp/intrusion/models/svc.pkl", bucket_name, f"{folder_path}/svc/model.pkl")
    
    
    
#     #Gradient Boost
#     start_train = time.time()
#     gbc = GradientBoostingClassifier()
#     gbc.fit(X_train, y_train)
#     end_train = time.time()
#     start_test = time.time()
#     y_pred5=gbc.predict(X_test)
#     end_test = time.time()
#     accuracy = accuracy_score(y_test,y_pred5)
#     f1 = f1_score(y_test, y_pred5)
#     precision = precision_score(y_test, y_pred5)
#     recall = (recall_score(y_test, y_pred5))
#     metrics.loc[len(metrics.index)] = [version, 'gbc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
#     with open('./tmp/intrusion/models/gbc.pkl', 'wb') as f:
#         pickle.dump(gbc, f)
#     s3_client.upload_file("tmp/intrusion/models/gbc.pkl", bucket_name, f"{folder_path}/gbc/model.pkl")
    
    
#     #Gaussian Naive Bayes
#     start_train = time.time()
#     gnb = GaussianNB()
#     gnb.fit(X_train, y_train)
#     end_train = time.time()
#     start_test = time.time()
#     y_pred6=gnb.predict(X_test)
#     end_test = time.time()
#     accuracy = accuracy_score(y_test,y_pred6)
#     f1 = f1_score(y_test, y_pred6)
#     precision = precision_score(y_test,y_pred6)
#     recall = recall_score(y_test, y_pred6)
#     metrics.loc[len(metrics.index)] = [version, 'gnb', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]  
#     with open('./tmp/intrusion/models/gnb.pkl', 'wb') as f:
#         pickle.dump(gnb, f)      
#     s3_client.upload_file("tmp/intrusion/models/gnb.pkl", bucket_name, f"{folder_path}/gnb/model.pkl")
    
    
    
#     #Artificial Neural Network
#     input_shape = [X_train.shape[1]]
#     start_train = time.time()
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(units=64, activation='relu', input_shape=input_shape),
#         tf.keras.layers.Dense(units=64, activation='relu'),
#         tf.keras.layers.Dense(units=1, activation='sigmoid')
#     ])
#     model.build()
#     model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
#     history = model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size=256, epochs=25)
#     end_train=time.time()
#     start_test = time.time()
#     y_pred7 = model.predict(X_test)
#     y_pred7 = (y_pred7 > 0.5).astype(np.int32)
#     end_test = time.time()
#     print(y_pred7)
#     accuracy = accuracy_score(y_test,y_pred7)
#     f1 = f1_score(y_test, y_pred7)
#     precision = precision_score(y_test,y_pred7)
#     recall = recall_score(y_test, y_pred7)
#     # accuracy = history.history['accuracy'][11]
#     metrics.loc[len(metrics.index)] = [version, 'ann', accuracy, f1, precision, recall, end_test-start_test, 0]
#     with open('./tmp/intrusion/models/ann.pkl', 'wb') as f:
#         pickle.dump(model, f)
#     s3_client.upload_file("tmp/intrusion/models/ann.pkl", bucket_name, f"{folder_path}/ann/model.pkl")

    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
        
    insert_query = """
        INSERT INTO intrusion_model_metrics (name, version, URI, in_use, accuracy, f1, precision, recall, train_time, test_time)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (name, version) DO NOTHING;
    """
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")

        # Iterate through DataFrame rows and insert into the table
        for index, row in metrics.iterrows():
            cursor.execute(insert_query, (
                row['Model'], 
                row['Version'], 
                f"s3://intrusionpipeline/version{version}/{row['Model']}/model.pkl", 
                False, 
                row['Accuracy'], 
                row['F1'], 
                row['Precision'], 
                row['Recall'], 
                row['Train_Time'], 
                row['Test_Time']
            ))
    
        conn.commit()
        print("Data inserted successfully.")

        cursor.close()
        conn.close()
        print("PostgreSQL connection closed.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL or insert data: {e}")
    

In [19]:
train_op()

Connected to PostgreSQL successfully.
version2/X_train.npy


/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Folder './tmp/intrusion/models' already exists.
Connected to PostgreSQL successfully.
Data inserted successfully.
PostgreSQL connection closed.


In [4]:
def model_eval_deploy() -> None:
    import pickle
    import pandas as pd
    import numpy as np 
    import json
    import os 
    import time
    import tensorflow as tf
    
    import boto3
    
    import psycopg2
    from psycopg2 import sql
    from sqlalchemy import create_engine
    
    from kubernetes import client 
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1SKLearnSpec
    
    def get_secret():

        secret_name = "DBCreds"
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            raise e

        secret = get_secret_value_response['SecretString']
    
        # Parse the secret string to get the credentials
        secret_dict = json.loads(secret)
        username = secret_dict['username']
        password = secret_dict['password']
        host = secret_dict['host']
        port = secret_dict['port']
        dbname = secret_dict['dbname']

        return username, password, host, port, dbname


    (user,pswd,host,port,db) = get_secret()
    
    
    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
    
    # Connect to PostgreSQL
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        exit()
        
    # Query to fetch data from the table
    try:
        fetch_query = "SELECT * FROM intrusion_model_metrics ORDER BY created_at desc LIMIT 1;"
        df = pd.read_sql(fetch_query, conn)
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    #print(f"df = {df['version'][0]}")
    
    if(not df.empty):
        version = df['version'][0]
    else:
        version = 1
        
    #print(f"version = {version}")
    
    try:
        fetch_query = f"SELECT * FROM intrusion_model_metrics where version={version} order by accuracy desc limit 1;"
        model = pd.read_sql(fetch_query, conn)
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    
    name = f"{model['name'][0]}-version{version}-id"
    print(name)
    namespace = utils.get_default_target_namespace()
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version
    
    isvc2 = V1beta1InferenceService(
        api_version=api_version,
        kind=constants.KSERVE_KIND,
        metadata=client.V1ObjectMeta(
            name=name,
            namespace=namespace,
            annotations={'sidecar.istio.io/inject': 'false'}
        ),
        spec=V1beta1InferenceServiceSpec(
            predictor=V1beta1PredictorSpec(
                service_account_name="s3-service-account",
                sklearn=V1beta1SKLearnSpec(
                    storage_uri=model['uri'][0]
                )
            )
        )
    )

    
    KServe = KServeClient()
    KServe.create(isvc2)
    
    try:
        fetch_query = "SELECT * FROM intrusion_model_metrics where in_use is true LIMIT 1;"
        old_model = pd.read_sql(fetch_query, conn)
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    update_query_new = """
        UPDATE intrusion_model_metrics
        SET in_use = true
        WHERE name = %s and version = %s;
    """
    
    update_query_old = """
        UPDATE intrusion_model_metrics
        SET in_use = false
        WHERE name = %s and version = %s;
    """
    try:
        cursor.execute(update_query_new, (model['name'][0], int(model['version'][0])))
        if(not old_model.empty):
            cursor.execute(update_query_old, (old_model['name'][0], int(old_model['version'][0])))
        conn.commit()
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    if(not old_model.empty):
        del_name = f"{old_model['name'][0]}-version{old_model['version'][0]}-id"
        namespace = utils.get_default_target_namespace()

        # Initialize the KServe client
        KServe = KServeClient()

        # Delete the inference service
        KServe.delete(del_name, namespace)
    
    
    cursor.close()
    conn.close()

In [21]:
model_eval_deploy()

Connected to PostgreSQL successfully.
rfc-version2


/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
import kfp
from kfp import dsl
from kfp import components

check_condition_op = components.func_to_container_op(func=check_condition, base_image='python:3.7', packages_to_install=['pandas==1.1.5', 'sqlalchemy==1.4.45', 'boto3', 'psycopg2-binary'])

read_csv_op = components.func_to_container_op(func=read_file, output_component_file='preprocess.yaml', base_image='python:3.7', packages_to_install=['pandas==1.1.5','scikit-learn==1.0.1', 'kfp', 'numpy', 'minio', 'psycopg2-binary', 'sqlalchemy==1.4.45','boto3'])

train_op = components.func_to_container_op(func=train_op, output_component_file='train.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn==1.0.1','numpy','minio', 'tensorflow', 'psycopg2-binary', 'sqlalchemy','boto3'])

eval_deploy = components.func_to_container_op(func=model_eval_deploy, output_component_file='eval_deploy.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn==1.0.1','numpy','minio', 'tensorflow', 'psycopg2-binary', 'sqlalchemy','boto3','kubernetes','kserve'])

read_data_op = kfp.components.load_component_from_file('preprocess.yaml')
train_op = kfp.components.load_component_from_file('train.yaml')
eval_deploy_op = kfp.components.load_component_from_file('eval_deploy.yaml')


def ml_pipeline():
    check_condition = check_condition_op()
    with dsl.Condition(check_condition.output == 'True'):
        preprocess = read_csv_op()
        train = train_op().after(preprocess)
        eval_deploy = eval_deploy_op().after(train)

# Compile the pipeline
kfp.compiler.Compiler().compile(ml_pipeline, 'intrusion_pipeline.yaml', disable_caching=True)